In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors

In [2]:
asdt=pd.read_csv('Breast Cancer(UCI)/Breast cancer data.csv')
pd.set_option('display.max_row',10)
pd.set_option('display.max_column',15)
asdt#['Class'].unique()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [3]:
pd.set_option('display.max_row',15)
asdt.isnull().any()

Sample code number             False
Clump Thickness                False
Uniformity of Cell Size        False
Uniformity of Cell Shape       False
Marginal Adhesion              False
Single Epithelial Cell Size    False
Bare Nuclei                    False
Bland Chromatin                False
Normal Nucleoli                False
Mitoses                        False
Class                          False
dtype: bool

In [4]:
asdt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
Sample code number             699 non-null int64
Clump Thickness                699 non-null int64
Uniformity of Cell Size        699 non-null int64
Uniformity of Cell Shape       699 non-null int64
Marginal Adhesion              699 non-null int64
Single Epithelial Cell Size    699 non-null int64
Bare Nuclei                    699 non-null object
Bland Chromatin                699 non-null int64
Normal Nucleoli                699 non-null int64
Mitoses                        699 non-null int64
Class                          699 non-null int64
dtypes: int64(10), object(1)
memory usage: 60.1+ KB


In [5]:
asdt.dtypes

Sample code number              int64
Clump Thickness                 int64
Uniformity of Cell Size         int64
Uniformity of Cell Shape        int64
Marginal Adhesion               int64
Single Epithelial Cell Size     int64
Bare Nuclei                    object
Bland Chromatin                 int64
Normal Nucleoli                 int64
Mitoses                         int64
Class                           int64
dtype: object

In [6]:
#asdt.apply(pd.to_numeric).info()
def cerr():
    ERV=set()
    ERVl=[]
    for x in asdt:
        for v in asdt[x]:
            try:
                val=float(v)
            except ValueError:
                ERVl.append(v)
                ERV.add(v)
    print(ERV,len(ERVl))
cerr()         

{'?'} 16


In [7]:
print(asdt.shape)
asdt = asdt[~(asdt == '?').any(axis=1)]
asdt.reset_index(drop=True,inplace=True)
print(asdt.shape)
cerr()

(699, 11)
(683, 11)
set() 0


c:\program files\python37\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [8]:
pd.set_option('display.max_column',15)
asdt.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [9]:
asdt.describe()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [10]:
print([asdt[x].nunique() for x in asdt])
asdt['Class'].value_counts(normalize=True)

[630, 10, 10, 10, 10, 10, 10, 10, 10, 9, 2]


2    0.650073
4    0.349927
Name: Class, dtype: float64

In [11]:
xdt,ydt=asdt.drop(['Class','Sample code number'],axis=1),asdt['Class']
x_train,x_vald,y_train,y_vald=train_test_split(xdt,ydt,test_size=.20)

In [12]:
pd.reset_option('display.max_row')
pd.reset_option('display.max_column')
pd.set_option('display.max_row',10)
pd.set_option('display.max_column',20)
xdt

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1
...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,3,2,1,1,1
679,2,1,1,1,2,1,1,1,1
680,5,10,10,3,7,3,8,10,2
681,4,8,6,4,3,4,10,6,1


In [13]:
clf=GaussianNB()
print(clf)

GaussianNB(priors=None, var_smoothing=1e-09)


In [14]:
clf_crs=cross_val_score(clf, x_train, y_train, cv=10)
print(clf_crs)
print('Average Score :',(sum(clf_crs)/10)*100)

[0.96363636 0.96363636 0.89090909 0.94545455 0.96363636 1.
 0.98181818 0.92592593 0.98148148 1.        ]
Average Score : 96.16498316498316


In [15]:
y_pred_crs=cross_val_predict(clf, x_train, y_train, cv=5)

In [16]:
confusion_matrix(y_train, y_pred_crs)

array([[341,  18],
       [  3, 184]], dtype=int64)

In [17]:
clf.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [18]:
y_pred=clf.predict(x_vald)

In [19]:
confusion_matrix( y_vald, y_pred)

array([[83,  2],
       [ 4, 48]], dtype=int64)

In [20]:
print('Precision :',precision_score(y_vald, y_pred,average=None),'\nRecall :',
      recall_score(y_vald, y_pred,average=None),'\n',
      '\nf1_Score',f1_score(y_vald, y_pred,average='macro'),'\nAccuracy :',accuracy_score(y_vald, y_pred)) #micro

Precision : [0.95402299 0.96      ] 
Recall : [0.97647059 0.92307692] 
 
f1_Score 0.9531463748290013 
Accuracy : 0.9562043795620438


In [21]:
clf_GNB=GaussianNB(priors=[0.655222, 0.344778]) #initial 0.655222, 0.344778 from data
print(clf_GNB)

GaussianNB(priors=[0.655222, 0.344778], var_smoothing=1e-09)


In [22]:
clf_GNB.fit(x_train, y_train)

GaussianNB(priors=[0.655222, 0.344778], var_smoothing=1e-09)

In [23]:
ym_pred=clf_GNB.predict(x_vald)

In [24]:
confusion_matrix( y_vald, ym_pred)

array([[83,  2],
       [ 4, 48]], dtype=int64)

In [25]:
print('Precision :',precision_score(y_vald, ym_pred,average=None),'\nRecall :',
recall_score(y_vald, ym_pred,average=None),'\nf1_Score :',f1_score(y_vald, ym_pred,average='macro'),
'\nAcurracy :',accuracy_score(y_vald, ym_pred)) #micro

Precision : [0.95402299 0.96      ] 
Recall : [0.97647059 0.92307692] 
f1_Score : 0.9531463748290013 
Acurracy : 0.9562043795620438


In [26]:
clf_GNB_g=GaussianNB()
print(clf_GNB_g)

GaussianNB(priors=None, var_smoothing=1e-09)


In [27]:
lprrs=[[1/i,1-1/i] for i in range(2,11)]
param_grid_GNB =[{"priors":lprrs}]
gridGNB = GridSearchCV(clf_GNB_g,param_grid_GNB,cv=10,scoring='precision_macro')  # scoring='precision_micro'      
gridGNB.fit(x_train,y_train)    
clf_GNB_g= gridGNB.best_estimator_                   # Best grid
print ('\n The best grid is as follows: \n')
print (clf_GNB_g)#grid.best_estimator_


 The best grid is as follows: 

GaussianNB(priors=[0.5, 0.5], var_smoothing=1e-09)


In [28]:
clf_GNB_g_crs=cross_val_score(clf_GNB_g, x_train, y_train, cv=10)
print(clf_GNB_g_crs)
print('Average Score :',(sum(clf_GNB_g_crs)/10)*100)

[0.96363636 0.96363636 0.89090909 0.94545455 0.96363636 1.
 0.98181818 0.92592593 0.98148148 1.        ]
Average Score : 96.16498316498316


In [29]:
yg_pred_crs=cross_val_predict(clf_GNB_g, x_train, y_train, cv=5)

In [30]:
confusion_matrix( y_train, yg_pred_crs)

array([[341,  18],
       [  3, 184]], dtype=int64)

In [31]:
clf_GNB_g.fit(x_train, y_train)

GaussianNB(priors=[0.5, 0.5], var_smoothing=1e-09)

In [32]:
yg_pred=clf_GNB_g.predict(x_vald)

In [33]:
confusion_matrix(y_vald, yg_pred)

array([[83,  2],
       [ 4, 48]], dtype=int64)

In [34]:
print('Precision :',precision_score(y_vald, yg_pred,average=None),'\nRecall :',
      recall_score(y_vald, yg_pred,average=None),'\nf1_Score :',f1_score(y_vald, yg_pred,average='macro'),
      '\nAcurracy :',accuracy_score(y_vald, yg_pred))#micro

Precision : [0.95402299 0.96      ] 
Recall : [0.97647059 0.92307692] 
f1_Score : 0.9531463748290013 
Acurracy : 0.9562043795620438


In [35]:
clf_MNB=MultinomialNB()
print(clf_MNB)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [36]:
clf_MNB_crs=cross_val_score(clf_MNB, x_train, y_train, cv=10)
print(clf_MNB_crs)
print('Average Score :',(sum(clf_MNB_crs)/10)*100)

[0.85454545 0.87272727 0.85454545 0.87272727 0.94545455 0.90909091
 0.92727273 0.92592593 0.94444444 0.94339623]
Average Score : 90.501302331491


In [37]:
y_pred_crs=cross_val_predict(clf_MNB, x_train, y_train, cv=5)

In [38]:
confusion_matrix(y_train, y_pred_crs)

array([[341,  18],
       [ 33, 154]], dtype=int64)

In [39]:
clf_MNB.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
y_pred=clf_MNB.predict(x_vald)

In [41]:
confusion_matrix( y_vald, y_pred)

array([[81,  4],
       [12, 40]], dtype=int64)

In [42]:
print('Precision :',precision_score(y_vald, y_pred,average=None),'\nRecall :',
    recall_score(y_vald, y_pred,average=None),'\n','\nf1_Score',f1_score(y_vald, y_pred,average='macro'),
    '\nAccuracy :',accuracy_score(y_vald, y_pred)) #micro

Precision : [0.87096774 0.90909091] 
Recall : [0.95294118 0.76923077] 
 
f1_Score 0.8717228464419475 
Accuracy : 0.8832116788321168


In [43]:
clf1_MNB=MultinomialNB(class_prior=[0.655222, 0.344778 ]) #initial 0.655222, 0.344778 from data
print(clf_MNB)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [44]:
clf1_MNB.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=[0.655222, 0.344778], fit_prior=True)

In [45]:
ym_pred=clf1_MNB.predict(x_vald)

In [46]:
confusion_matrix( y_vald, ym_pred)

array([[81,  4],
       [12, 40]], dtype=int64)

In [47]:
print('Precision :',precision_score(y_vald, ym_pred,average=None),'\nRecall :',
      recall_score(y_vald, ym_pred,average=None),'\nf1_Score :',f1_score(y_vald, ym_pred,average='macro'),
      '\nAcurracy :',accuracy_score(y_vald, ym_pred)) #micro

Precision : [0.87096774 0.90909091] 
Recall : [0.95294118 0.76923077] 
f1_Score : 0.8717228464419475 
Acurracy : 0.8832116788321168


In [48]:
clf_MNB_g=MultinomialNB()
print(clf_MNB_g)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [49]:
alf=[i for i in range(2,10)]
#alf=[1-1/i for i in range(2,10)]
lprrs=[[1/i,1-1/i] for i in range(2,11)]
ftprr=[True,False]
param_grid_MNB =[{'alpha':alf,"class_prior":lprrs,'fit_prior':ftprr}]
gridMNB = GridSearchCV(clf_MNB_g,param_grid_MNB,cv=10)  # scoring='precision_micro' #,scoring='precision_macro'     
gridMNB.fit(x_train,y_train)    
clf_MNB_g= gridMNB.best_estimator_                   # Best grid
print ('\n The best grid is as follows: \n')
print (clf_MNB_g)#grid.best_estimator_


 The best grid is as follows: 

MultinomialNB(alpha=2, class_prior=[0.5, 0.5], fit_prior=True)


c:\program files\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [50]:
clf_MNB_g_crs=cross_val_score(clf_MNB_g, x_train, y_train, cv=10)
print(clf_MNB_g_crs)
print('Average Score :',(sum(clf_MNB_g_crs)/10)*100)

[0.83636364 0.83636364 0.8        0.83636364 0.89090909 0.8
 0.92727273 0.81481481 0.87037037 0.86792453]
Average Score : 84.803824407598


In [51]:
yg_pred_crs=cross_val_predict(clf_MNB_g, x_train, y_train, cv=5)

In [52]:
confusion_matrix( y_train, yg_pred_crs)

array([[304,  55],
       [ 27, 160]], dtype=int64)

In [53]:
print('Precision :',precision_score(y_vald, yg_pred,average=None),'\nRecall :',
      recall_score(y_vald, yg_pred,average=None),'\nf1_Score :',f1_score(y_vald, yg_pred,average='macro'),
      '\nAcurracy :',accuracy_score(y_vald, yg_pred))#micro

Precision : [0.95402299 0.96      ] 
Recall : [0.97647059 0.92307692] 
f1_Score : 0.9531463748290013 
Acurracy : 0.9562043795620438


In [54]:
clf_KN=KNeighborsClassifier()
print(clf_KN)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')


In [55]:
clf_KN_crs=cross_val_score(clf_KN, x_train, y_train, cv=10)
print(clf_KN_crs)
print('Average Score :',(sum(clf_KN_crs)/10)*100)

[0.94545455 0.98181818 0.92727273 0.98181818 1.         1.
 0.94545455 0.94444444 1.         1.        ]
Average Score : 97.26262626262626


In [56]:
y_pred_crs=cross_val_predict(clf_KN, x_train, y_train, cv=5)

In [57]:
confusion_matrix(y_train, y_pred_crs)

array([[352,   7],
       [  6, 181]], dtype=int64)

In [58]:
clf_KN.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [59]:
y_pred=clf_KN.predict(x_vald)

In [60]:
confusion_matrix( y_vald, y_pred)

array([[84,  1],
       [ 7, 45]], dtype=int64)

In [61]:
print('Precision :',precision_score(y_vald, y_pred,average=None),'\nRecall :',
        recall_score(y_vald, y_pred,average=None),'\n','\nf1_Score',f1_score(y_vald, y_pred,average='macro'),
        '\nAccuracy :',accuracy_score(y_vald, y_pred)) #micro

Precision : [0.92307692 0.97826087] 
Recall : [0.98823529 0.86538462] 
 
f1_Score 0.936456400742115 
Accuracy : 0.9416058394160584


In [62]:
clf1_KN=KNeighborsClassifier(n_neighbors=10, metric='hamming') #'euclidean','hamming','canberra','cosine'
print(clf1_KN)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='hamming',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')


In [63]:
clf1_KN.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='hamming',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [64]:
ym_pred=clf1_KN.predict(x_vald)

In [65]:
confusion_matrix( y_vald, ym_pred)

array([[85,  0],
       [14, 38]], dtype=int64)

In [66]:
print('Precision :',precision_score(y_vald, ym_pred,average=None),'\nRecall :',
      recall_score(y_vald, ym_pred,average=None),'\nf1_Score :',f1_score(y_vald, ym_pred,average='macro'),
      '\nAcurracy :',accuracy_score(y_vald, ym_pred)) #micro

Precision : [0.85858586 1.        ] 
Recall : [1.         0.73076923] 
f1_Score : 0.8841787439613527 
Acurracy : 0.8978102189781022


In [67]:
clf_KN_g=KNeighborsClassifier(algorithm="auto")
print(clf_KN_g)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')


In [69]:
nbdn=[i for i in range(3,20)]
nbdlf=[i for i in range(15, 45)]
mtrc=['minkowski','euclidean','hamming','canberra']#'cosine'
#algo=['auto', 'ball_tree', 'kd_tree', 'brute']
dtyp=[2,3]
param_grid_KN =[{'leaf_size':nbdlf,'metric':mtrc,'n_neighbors':nbdn,'p':dtyp}]
gridKN = GridSearchCV(clf_KN_g, param_grid_KN, cv=10, scoring='precision_macro')  # scoring='precision_micro'      
gridKN.fit(x_train,y_train)    
clf_KN_g= gridKN.best_estimator_                   # Best grid
print ('\n The best grid is as follows: \n')
print (clf_KN_g)#grid.best_estimator_


 The best grid is as follows: 

KNeighborsClassifier(algorithm='auto', leaf_size=16, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')


In [70]:
clf_KN_g_crs=cross_val_score(clf_KN_g, x_train, y_train, cv=10)
print(clf_KN_g_crs)
print('Average Score :',(sum(clf_KN_g_crs)/10)*100)

[0.96363636 0.98181818 0.92727273 0.98181818 1.         1.
 0.98181818 0.94444444 1.         1.        ]
Average Score : 97.80808080808082


In [71]:
yg_pred_crs=cross_val_predict(clf_KN_g, x_train, y_train, cv=5)

In [72]:
confusion_matrix(y_train, y_pred_crs)

array([[352,   7],
       [  6, 181]], dtype=int64)

In [73]:
clf_KN_g.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=16, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')

In [74]:
yg_pred=clf_KN_g.predict(x_vald)

In [75]:
confusion_matrix(y_vald, yg_pred)

array([[84,  1],
       [ 8, 44]], dtype=int64)

In [76]:
print('Precision :',precision_score(y_vald, yg_pred,average=None),'\nRecall :',
      recall_score(y_vald, yg_pred,average=None),'\nf1_Score :',f1_score(y_vald, yg_pred,average='macro'),
      '\nAcurracy :',accuracy_score(y_vald, yg_pred))#micro

Precision : [0.91304348 0.97777778] 
Recall : [0.98823529 0.84615385] 
f1_Score : 0.9281845186091211 
Acurracy : 0.9343065693430657


In [77]:
#print('Precision :',precision_score(),'\nRecall :',recall_score(),
#      '\nf1_Score :',f1_score(,average='macro'),'\nAcurracy :',accuracy_score())#micro

In [78]:
#print('Precision :',,'\nRecall :',,'\nf1_Score :',,'\nAcurracy :',)

In [79]:
#print('Precision :',precision_score(y_vald, y_pred,average=None, pos_label="neg"),'\nRecall :',
#recall_score(y_vald, y_pred,average=None, pos_label="neg"),'\n','\nf1_Score',f1_score(y_vald, y_pred,average='macro'),
#'\nAccuracy :',accuracy_score(y_vald, y_pred)) #micro

#:\program files\python37\lib\site-packages\sklearn\metrics\classification.py:1052: UserWarning: Note that pos_label
#(set to 'neg') is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive
#class. % (pos_label, average), UserWarning)

In [80]:
#param_grid =[{'n_neighbors':K},{'metric':['euclidean','minkowski','hamming','canberra']}]
#grid = grid_search.GridSearchCV(svr,param_grid,cv=10,scoring='precision_macro')          
#grid.fit(trn_data,class_label_trn)    
#clf= grid.best_estimator_                   # Best grid
#print ('\n The best grid is as follows: \n')
#print (grid.best_estimator_)